In [128]:
from bs4 import BeautifulSoup
import requests
from requests import Request, Session
import pandas as pd
import re
from ftfy import fix_text

## Добро Mail

In [129]:
def dobrma(org_name):
    org_name = org_name.replace(' ', '+')
    urls = f'https://dobro.mail.ru/funds/search/?query={org_name}&recipient=all&city=any'
    response = requests.get(urls)
    soup = BeautifulSoup(response.text)
    req = soup.find_all('div', class_='cols__column cols__column_small_percent-100 cols__column_medium_percent-50 cols__column_large_percent-50')
    if len(req) != 0:
        for i in range(len(req)):
            name = str(req[i].find('span', 'link__text'))
            name = name.replace('<span class="link__text">', '')
            name = name.replace('</span>', '')
            url = str(req[i].find('a', 'link link_font_large margin_bottom_10 link-holder'))
            url = url.replace('<a class="link link_font_large margin_bottom_10 link-holder" href="', 'https://dobro.mail.ru')
            url = url.split('"')[0]
            city = str(req[i].find('div', 'p-fund__city margin_bottom_5'))
            city = city.replace('<div class="p-fund__city margin_bottom_5">', '')
            city = city.replace('</div>', '')
            responses = requests.get(url)
            soups = BeautifulSoup(responses.text)
            site = soups.find_all('div', class_='p-fund-detail__info-row')
            if str(site[0].find('a', 'link')) == None: site = str(site[0].find('a', 'link')); site = site.replace('<span class="link__text">', ''); site = site.replace('</span>', '')
            else: site = str(site[1].find('a', 'link')); site = site.replace('<span class="link__text">', ''); site = site.split('"')[3]
        return site
    else: 
        return False

def isin_dobrmail(org_name):
    if dobrma(org_name):
        return f'Организация {org_name} присутствует на сайте https://dobro.mail.ru/'
    else:
        return f'Организация {org_name} отсутствует на сайте https://dobro.mail.ru/'

In [130]:
org_name = "«Подсолнух»"

isin_dobrmail(org_name)

'Организация «Подсолнух» присутствует на сайте https://dobro.mail.ru/'

## Все вместе

In [110]:
def return_vse_meste_orgs():
    urls2 = 'https://wse-wmeste.ru/about/uchastniki-assotsiatsii/'

    vse_vmest = []
    desc = []
    response2 = requests.get(urls2)
    soup3 = BeautifulSoup(response2.text)
    query3 = soup3.find('div', 'col-md-12')
    query3 = query3.find_all('div', 'wp-block-themeisle-blocks-posts-grid-post-blog wp-block-themeisle-blocks-posts-grid-post-plain')

    for i in range(len(query3)):
        name = str(query3[i].find('h5', 'wp-block-themeisle-blocks-posts-grid-post-title text-center')).split('>')[2].replace('</a', '')
        disc = str(query3[i].find('p', 'wp-block-themeisle-blocks-posts-grid-post-description')).replace('<p class="wp-block-themeisle-blocks-posts-grid-post-description">', '').replace('</p>', '')
        vse_vmest.append(' '.join(re.findall('\w+', fix_text(name.lower()).replace('\xa0', ' ')))) 
        desc.append(fix_text(disc).replace('\xa0', ' '))
    return vse_vmest

In [115]:
def find_in_vse_vmeste(name_org, all_orgs):
    name_org = ' '.join(re.findall(r'\w+', name_org.lower()))
    if name_org in all_orgs:
        return f'Организация "{name_org.capitalize()}" есть на сайте https://wse-wmeste.ru/about/uchastniki-assotsiatsii/'
    else: 
        return f'Организация "{name_org.capitalize()}" отсутствует на сайте https://wse-wmeste.ru/about/uchastniki-assotsiatsii/'

In [135]:
name_org = 'Здесь и сейчас'

vse_vmeste_orgs = return_vse_meste_orgs()
find_in_vse_vmeste(name_org, vse_vmeste_orgs)

'Организация "Здесь и сейчас" есть на сайте https://wse-wmeste.ru/about/uchastniki-assotsiatsii/'

## Нужна помощь

In [125]:
org_name = "спбгу"

def find_nuzna_pomosh(org_name):
    org_name = ' '.join(re.findall('\w+', org_name))
    url = 'https://nuzhnapomosh.ru/wp-content/plugins/nuzhnapomosh/funds.php'
    typ = 'POST'
    req = Request(typ, url, files = {'np_name': (None, org_name)}).prepare()
    s = Session()
    respo = s.send(req);respo.text
    soup2 = BeautifulSoup(respo.text)
    quotes2 = soup2.find_all('div', class_='np-card__inner')

    if len(quotes2) != 0:
        for i in range(len(quotes2)):
            name = str(quotes2[i].find('h4', 'np-card__title'));  name = name.replace('<h4 class="np-card__title">', '');  name = name.replace('</h4>', '')
            descr = str(quotes2[i].find('p', 'np-card__descr'));  descr = descr.replace('<p class="np-card__descr">', '');  descr = descr.replace('</p>', '')
            money = str(quotes2[i].find('li', 'np-card__row'));  money = money.split('<span>')[1];  money = money.replace('</li>', '');  money = money.replace(' ₽</span>', '');money = money.replace('\n', '')
            site = str(quotes2[i].find('a', 'np-card__link'));  site = site.replace('<a class="np-card__link" href="', '');  site = site.split('"')[0]
        return (f'Организация "{name}" найдена на сайте https://nuzhnapomosh.ru{site}',
                'Описание фонда: ' + descr, 
                'Деньги в фонде: ' + money + ' рублей')
    else: 
        return f'{org_name} не найдена https://nuzhnapomosh.ru'

In [126]:
find_nuzna_pomosh(org_name)

('Организация "Ассоциация выпускников СПбГУ" найдена на сайте https://nuzhnapomosh.ru/funds/associaciya-vypusknikov-spbgu/',
 'Описание фонда: Объединяет всех, кто учился в первом Университете России',
 'Деньги в фонде: 431 579 рублей')

In [136]:
def find_in_reestr_sonko(inn):
    reestr_sonko = pd.read_excel('reestr_SONKO.xlsx')
    if inn in reestr_sonko['ИНН'].unique():
        status = reestr_sonko.loc[reestr_sonko['ИНН'] == inn, 'Текущий статус'].values[0]
        return f'Найдено в реестре социально значимых НКО, текущий статус: {status.lower()}'
    else:
        return 'Не найдено в реестре социально значимых НКО'

In [140]:
find_in_reestr_sonko(3255990127)

'Найдено в реестре социально значимых НКО, текущий статус: действующая'

In [141]:
reestr_sonko = pd.read_excel('reestr_SONKO.xlsx')

[3255990127,
 7325024705,
 6501173421,
 2723121547,
 323361766,
 323372704,
 5837038560,
 326009168,
 1832054391,
 5401301011,
 274019998,
 1434006663,
 1829005370,
 3015029833,
 3123018268,
 3109002139,
 4401020481,
 4826018506,
 5753028460,
 5501023230,
 5836115130,
 5902703018,
 6661003668,
 6453039984,
 6624001800,
 6731017138,
 7106038617,
 7226003084,
 7202089854,
 7604021895,
 8610007611,
 3717002896,
 3801017820,
 3904021899,
 4709008078,
 3729010500,
 7106042469,
 7227003270,
 8300003286,
 714006512,
 4720027050,
 7536056343,
 3123164332,
 3438007580,
 5501111656,
 601020523,
 5944206628,
 3702634300,
 4414014157,
 1402999017,
 1402999049,
 2461195034,
 2983998523,
 1435970327,
 4345299156,
 4826090171,
 4401115408,
 2634091121,
 7206046883,
 7326999038,
 8901998149,
 725990884,
 3126995014,
 3123232494,
 4705470644,
 7536986606,
 725991278,
 2543971376,
 2635827990,
 2901992315,
 7328999121,
 7203339803,
 223004680,
 6230091677,
 6154142630,
 1102077741,
 3123392530,
 4632219

In [146]:
l = list(reestr_sonko['ИНН'].unique())

strl = []

for i in l:
    strl.append(str(i))

with open("sonko.py", "w") as outfile:
    outfile.write("sonko = [")
    outfile.write("\n,".join(strl))
    outfile.write("]")